<a href="https://colab.research.google.com/github/amadoujr/MultimodalRAG-Chatbot-Using-colpali/blob/main/RAG_Conversation_Health.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### packages installation

In [ ]:
!pip install --quiet --upgrade  langchain langchain-community gradio   #langchainhub
!pip install pypdf -q
!sudo apt-get install -y poppler-utils --quiet
!pip install --quiet --upgrade byaldi

In [ ]:
import os
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import gradio as gr
import csv
from langchain.chains import create_retrieval_chain,create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.retrievers import BaseRetriever
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage,SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.language_models.llms import LLM
from byaldi import RAGMultiModalModel
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
import os
import base64

**Google model for the generation**

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

### Loading a model with byaldi


In [ ]:
# Optionally, you can specify an `index_root`, which is where it'll save the index. It defaults to ".byaldi/".
rag_model = RAGMultiModalModel.from_pretrained("vidore/colqwen2-v1.0")

Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

### Indexing our data

In [ ]:
def index_model(index_name,path_file):
    index_name = index_name
    rag_model.index(
    input_path=path_file, # The path to your documents
    index_name=index_name, # The name you want to give to your index. It'll be saved at `index_root/index_name/`.
    store_collection_with_index=True, # Whether the index should store the base64 encoded documents.
    overwrite=True # Whether to overwrite an index if it already exists. If False, it'll return None and do nothing if `index_root/index_name` exists.
    )

In [ ]:
index_model("cv_data", "/content/Amadou_Barro_CV.pdf")

Added page 1 of document 0 to index.
Index exported to .byaldi/cv_data
Index exported to .byaldi/cv_data


In [ ]:
text_query = "A qui appartient ce CV?"
rag_model.search(text_query, k=1)

#### Integrate ColPali (Byaldi) with LangChain to create the retriever

In [ ]:
class ByaldiRetriever(BaseRetriever):

    def _get_relevant_documents(self, query: str):
        k: int = 1
        # Use Byaldi to search for relevant documents
        results = rag_model.search(query, k=k)

        # Convert results into LangChain's Document format
        documents = []
        for result in results:
            doc = Document(
                page_content=result["base64"],
                metadata={
                    "doc_id": result["doc_id"],
                    "page_num": result["page_num"],
                    "score": result["score"],
                }
            )
            documents.append(doc)
        return documents

retriever = ByaldiRetriever()

### Retriever and Generation

In [ ]:
### Contextualize question ###

contextualize_q_system_prompt = (
    "Compte tenu de l'historique de la conversation et de la dernière question "
    "de l'utilisateur, reformule une question autonome compréhensible sans l'historique. "
    "Ne réponds pas à la question, reformule-la uniquement si nécessaire."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

### Answer question ###
system_prompt = (
    "Tu es un assistant pour les tâches de question-réponse. "
    "Aide toi uniquement du contexte, qui est ici une image pour répondre aux questions que je vais te poser."
    "Si le contexte ne fourni pas assez d'élément mais que tu connais la réponse alors donne la"
    "Si le contexte n'est pas assez fourni et que tu ne sais pas, dis que tu ne sais pas "
)

user_prompt = [
                {"type": "text",
                 "text": "Voici la question : {input}"
                },
                {"type": "image_url",
                "image_url": {"url": "data:image/jpeg;base64,{context}"}
                },
              ]

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("user", user_prompt),
    ],
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [ ]:
chat_history = []
question = "Qu'est ce qu'il cherche?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
ai_msg_1["answer"]

"D'après son CV, Amadou Barro cherche un stage de fin d'études en data science, data engineering ou data analyst (mars-septembre)."

In [ ]:
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "Dans quoi t'as dis?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})
ai_msg_2["answer"]

"J'ai dit qu'il cherchait un stage de fin d'études en data science, data engineering ou data analyst."

In [ ]:
def save_chat_history(history, filename="chat_history.csv"):
  with open(filename, "w+", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Role", "Content"])  # En-têtes de colonnes
    for turn in history:
      if isinstance(turn, HumanMessage):
          writer.writerow(["user", turn.content])
      elif isinstance(turn, AIMessage):
          writer.writerow(["assistant", turn.content])

In [ ]:
chat_history = []

def respond(message, history):
  global chat_history
  for msg in history:
    if msg['role'] == "user":
      chat_history.append(HumanMessage(content=msg['content']))
    elif msg['role'] == "assistant":
      chat_history.append(AIMessage(content=msg['content']))

  chat_history.append(HumanMessage(content=message))
  ai_message = rag_chain.invoke({
      "input": message,
      "chat_history": chat_history
  })
  chat_history.append(AIMessage(content=ai_message["answer"]))
  save_chat_history(chat_history)
  return ai_message["answer"]


In [ ]:
def create_index(file, index_name):
    index_name = str(index_name)
    try:
        # Check if files were uploaded
        if not file:
            return "Aucun fichier téléchargé. Veuillez télécharger un fichier PDF."

        file_path = file.name
        if not file_path.endswith(".pdf"):
            return "Le fichier téléchargé n'est pas un PDF valide."

        # Create the index
        index_model(index_name, file_path)
        return f"Index '{index_name}' créé avec succès !"
    except Exception as e:
        return f"Erreur lors de la création de l'index : {str(e)}"

 # Function to clear chat history
def clear_chat():
  global chat_history
  chat_history = []  # Reset the chat history
  return []  # Return an empty list for the Chatbot component

In [ ]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Assistant Chatbot AI")
    msg = gr.Textbox(placeholder="Entrez votre message ici...")
    clear = gr.ClearButton([msg, chatbot])
    export_btn = gr.Button("Exporter l'historique")

    # File upload component
    file_upload = gr.File(label="Upload a PDF or folder to create an index")
    index_name_input = gr.Textbox(label="Nom de l'index", placeholder="Entrez un nom pour l'index")
    create_index_btn = gr.Button("Créer l'index")
    index_status = gr.Textbox(label="Statut de l'index", interactive=False)

    def user(user_message, history):
      history.append([user_message, None])
      return "", history

    def bot(history):
      global chat_history
      formatted_history = []
      for turn in history[:-1]:
        formatted_history.append({"role": "user", "content": turn[0]})
        if turn[1]:
          formatted_history.append({"role": "assistant", "content": turn[1]})
      bot_message = respond(history[-1][0], formatted_history)
      history[-1][1] = bot_message
      chat_history.append({"role": "user", "content": history[-1][0]})
      chat_history.append({"role": "assistant", "content": bot_message})
      return history


    def export_history():
      global chat_history
      save_chat_history(chat_history, filename="chat_history.csv")
      return "Historique exporté avec succès !"

     # Create index button callback
    create_index_btn.click(
        create_index,
        inputs=[file_upload, index_name_input],
        outputs=[index_status]
    )

    clear.click(clear_chat, outputs=[chatbot])

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    export_btn.click(export_history, outputs=[gr.Textbox(label="Statut d'exportation")])

demo.launch(debug=True)

<ipython-input-21-23f23c121b4b>:2: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Assistant Chatbot AI")


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b6f3c0d3ac84432d73.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b6f3c0d3ac84432d73.gradio.live
